In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import math
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
%matplotlib inline

# Deal with data

## Import dataframe

In [3]:
pd.set_option('display.max_columns',None)

In [4]:
df = pd.read_csv('fifa21_train.csv')
display(df.head())
df.shape

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,Right,1,"Jul 1, 2015",NaN,€525K,€4K,€801K,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,Right,0,"Jan 16, 2015","Dec 31, 2020",€8.5M,€23K,€0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,Right,0,"Jan 31, 2019",NaN,€9M,€49K,€15.3M,2019 ~ 2022,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,73,73+2,73+2,73+2,80+0,79+0,79+0,79+0,80+0,80+0,80+0,80+0,79+1,74+2,74+2,74+2,79+1,59+2,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,Right,13,"Jul 1, 2016",NaN,€275K,€4K,€694K,2016 ~ 2021,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,7,50+2,50+2,50+2,51+0,51+0,51+0,51+0,51+0,53+2,53+2,53+2,53+2,56+2,56+2,56+2,53+2,56+2,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,Right,8,"Jul 1, 2018",NaN,€725K,€2K,€1.4M,2018 ~ 2021,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,4,56+2,56+2,56+2,59+0,59+0,59+0,59+0,59+0,61+2,61+2,61+2,62+2,63+2,63+2,63+2,62+2,64+2,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


(11701, 101)

In [5]:
list(df.columns)

['ID',
 'Name',
 'Age',
 'Nationality',
 'Club',
 'BP',
 'Position',
 'Team & Contract',
 'Height',
 'Weight',
 'foot',
 'Growth',
 'Joined',
 'Loan Date End',
 'Value',
 'Wage',
 'Release Clause',
 'Contract',
 'Attacking',
 'Crossing',
 'Finishing',
 'Heading Accuracy',
 'Short Passing',
 'Volleys',
 'Skill',
 'Dribbling',
 'Curve',
 'FK Accuracy',
 'Long Passing',
 'Ball Control',
 'Movement',
 'Acceleration',
 'Sprint Speed',
 'Agility',
 'Reactions',
 'Balance',
 'Power',
 'Shot Power',
 'Jumping',
 'Stamina',
 'Strength',
 'Long Shots',
 'Mentality',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Defending',
 'Marking',
 'Standing Tackle',
 'Sliding Tackle',
 'Goalkeeping',
 'GK Diving',
 'GK Handling',
 'GK Kicking',
 'GK Positioning',
 'GK Reflexes',
 'Total Stats',
 'Base Stats',
 'W/F',
 'SM',
 'A/W',
 'D/W',
 'IR',
 'PAC',
 'SHO',
 'PAS',
 'DRI',
 'DEF',
 'PHY',
 'Hits',
 'LS',
 'ST',
 'RS',
 'LW',
 'LF',
 'CF',
 'RF',
 'RW',
 'LAM'

## Standardize Headers

In [6]:
def stHead(df):
    new_header = []
    for h in df.columns:
        x = h.lower().replace(' ','_')
        new_header.append(x)
    df.columns = new_header
    display(df)

In [7]:
stHead(df)

,id,name,age,nationality,club,bp,position,team_&_contract,height,weight,foot,growth,joined,loan_date_end,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,Right,1,"Jul 1, 2015",NaN,€525K,€4K,€801K,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,Right,0,"Jan 16, 2015","Dec 31, 2020",€8.5M,€23K,€0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,Right,0,"Jan 31, 2019",NaN,€9M,€49K,€15.3M,2019 ~ 2022,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,73,73+2,73+2,73+2,80+0,79+0,79+0,79+0,80+0,80+0,80+0,80+0,79+1,74+2,74+2,74+2,79+1,59+2,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,Right,13,"Jul 1, 2016",NaN,€275K,€4K,€694K,2016 ~ 2021,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,7,50+2,50+2,50+2,51+0,51+0,51+0,51+0,51+0,53+2,53+2,53+2,53+2,56+2,56+2,56+2,53+2,56+2,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,Right,8,"Jul 1, 2018",NaN,€725K,€2K,€1.4M,2018 ~ 2021,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,4,56+2,56+2,56+2,59+0,59+0,59+0,59+0,59+0,61+2,61+2,61+2,62+2,63+2,63+2,63+2,62+2,64+2,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,LB,Jagiellonia Białystok 2018 ~ 2021,"6'1""",168lbs,Left,5,"Feb 8, 2018",NaN,€625K,€3K,€891K,2018 ~ 2021,224,62,24,51,59,28.0,248,57,49.0,29,54,59,333,78,73,68.0,53,61.0,273,31,72.0,77,68,25,266,63,59.0,51.0,54.0,39,46.0,184,59,64,61.0,61,13,14,14,6,14,1589,347,3 ★,2★,Medium

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11701 entries, 0 to 11700
Columns: 101 entries, id to ova
dtypes: float64(10), int64(43), object(48)
memory usage: 9.0+ MB


In [9]:
df.describe()

,id,age,growth,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,pac,sho,pas,dri,def,phy,ova
count,11701.000000,11701.00000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11660.000000,11701.000000,11701.000000,11660.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11660.000000,11701.000000,11660.000000,11701.000000,11701.000000,11660.000000,11701.000000,11701.00000,11701.000000,11701.000000,11701.000000,11694.000000,11694.000000,11660.000000,11701.000000,11422.000000,11701.000000,11701.000000,11701.000000,11660.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000
mean,219436.649004,25.27049,5.534655,258.482181,51.593795,48.048116,53.494060,60.436800,45.067324,266.497222,57.852491,49.617839,44.358260,54.214683,60.627810,322.617383,65.432527,65.446628,64.582075,62.899838,64.709348,302.083326,59.726348,65.036621,63.214939,65.17084,49.162465,261.704982,56.939065,46.815119,52.485890,55.439194,50.279378,59.947732,140.674558,46.964875,48.042560,45.827702,77.726946,15.604906,15.517734,15.494402,15.543458,15.768310,1629.786599,361.106316,68.095206,55.046406,58.893257,64.223229,50.030254,64.817964,66.921972
std,37480.006004,4.95764,5.810903,72.267012,17.872747,19.399617,16.956195,13.971811,17.732902,76.835287,18.050938,18.180408,17.484142,14.886875,15.886688,55.330054,14.735329,14.516404,14.484351,9.020199,14.131318,49.697749,13.511767,12.017118,15.350014,12.76541,19.058991,62.935298,17.066535,20.808153,19.069465,13.695285,15.713965,11.769872,61.599006,20.190466,21.470841,21.217062,81.221525,16.850815,16.155245,16.005114,16.409881,17.151738,260.766470,40.318331,11.247282,13.851853,10.174939,9.818311,16.841283,9.757873,6.876081
min,27.000000,16.00000,-1.000000,33.000000,6.000000,3.000000,5.000000,8.000000,4.000000,43.000000,5.000000,4.000000,5.000000,9.000000,5.000000,113.000000,13.000000,11.000000,14.000000,24.000000,17.000000,133.000000,12.000000,25.000000,12.000000,20.00000,4.000000,55.000000,9.000000,5.000000,2.000000,10.000000,8.000000,12.000000,17.000000,3.000000,6.000000,6.000000,12.000000,1.000000,2.000000,2.000000,2.000000,1.000000,731.000000,228.000000,26.000000,17.000000,25.000000,28.000000,12.000000,27.000000,38.000000
25%,204060.000000,21.00000,0.000000,232.000000,41.000000,33.000000,46.000000,56.000000,32.000000,232.000000,53.000000,37.000000,32.000000,45.000000,57.000000,294.000000,58.000000,59.000000,57.000000,57.000000,57.000000,271.000000,50.000000,58.000000,56.000000,58.00000,35.000000,235.000000,45.000000,26.000000,43.000000,47.000000,40.000000,53.000000,83.000000,29.000000,28.000000,25.000000,48.000000,8.000000,8.000000,8.000000,8.000000,8.000000,1490.000000,333.000000,62.000000,46.000000,52.000000,59.000000,35.000000,59.000000,62.000000
50%,229040.000000,25.00000,4.000000,271.000000,56.000000,52.000000,56.000000,63.000000,47.000000,279.000000,63.000000,52.000000,43.000000,57.000000,64.000000,331.000000,68.000000,68.000000,67.000000,63.000000,67.000000,308.000000,61.000000,66.000000,66.000000,66.00000,54.000000,269.000000,60.000000,52.000000,57.000000,57.000000,51.000000,61.000000,156.000000,52.000000,55.000000,52.000000,53.000000,11.000000,11.000000,11.000000,11.000000,11.000000,1659.000000,362.000000,69.000000,58.000000,59.000000,65.000000,53.000000,66.000000,67.000000
75%,244176.000000,29.00000,10.000000,306.000000,65.000000,64.000000,65.000000,69.000000,59.000000,319.000000,70.000000,64.000000,58.00

## Identify numerical and categorical variables

In [10]:
list(df.dtypes)

[dtype('int64'),
 dtype('O'),
 dtype('int64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('int64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dt

In [11]:
df._get_numeric_data()

,id,age,growth,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,pac,sho,pas,dri,def,phy,ova
0,184383,26,1,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,69,51,63,63,51,60,64
1,188044,30,0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,83,75,68,82,33,71,77
2,184431,33,0,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,80,77,78,86,27,56,80
3,233796,22,13,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,57,44,54,57,57,60,59
4,234799,23,8,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,66,44,60,64,60,66,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,25,5,224,62,24,51,59,28.0,248,57,49.0,29,54,59,333,78,73,68.0,53,61.0,273,31,72.0,77,68,25,266,63,59.0,51.0,54.0,39,46.0,184,59,64,61.0,61,13,14,14,6,14,1589,347,75,28,56,59,60,69,65
11697,214680,27,4,84,12,14,13,29,16.0,96,17,15.0,19,27,18,237,40,40,38.0,65,54.0,217,50,59.0,29,64,15,101,30,22.0,15.0,15.0,19,56.0,41,12,16,13.0,347,72,67,66,70,72,1123,387,72,67,66,72,40,70,70
11698,221489,22,8,286,64,66,51,64,41.0,291,71,57.0,38,58,67,388,77,76,86.0,62,87.0,278,61,71.0,64,37,45,241,27,25.0,66.0,58.0,65,58.0,62,37,12,13.0,57,13,10,11,7,16,1603,337,76,60,60,72,26,43,67
11699,146717,26,7,254,64,66,51,73,NaN,284,74,NaN,63,72,75,198,71,72,NaN,55,NaN,258,61,NaN,71,64,62,241,64,61.0,63.0,NaN,53,NaN,113,55,58,NaN,81,9,24,72,24,24,1429,400,72,63,70,72,57,66,68


In [12]:
numeCols = list(df._get_numeric_data().columns)
numeCols

['id',
 'age',
 'growth',
 'attacking',
 'crossing',
 'finishing',
 'heading_accuracy',
 'short_passing',
 'volleys',
 'skill',
 'dribbling',
 'curve',
 'fk_accuracy',
 'long_passing',
 'ball_control',
 'movement',
 'acceleration',
 'sprint_speed',
 'agility',
 'reactions',
 'balance',
 'power',
 'shot_power',
 'jumping',
 'stamina',
 'strength',
 'long_shots',
 'mentality',
 'aggression',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'composure',
 'defending',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'goalkeeping',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes',
 'total_stats',
 'base_stats',
 'pac',
 'sho',
 'pas',
 'dri',
 'def',
 'phy',
 'ova']

In [13]:
df.select_dtypes('object')

,name,nationality,club,bp,position,team_&_contract,height,weight,foot,joined,loan_date_end,value,wage,release_clause,contract,w/f,sm,a/w,d/w,ir,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,A. Pasche,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,Right,"Jul 1, 2015",NaN,€525K,€4K,€801K,2015 ~ 2020,4 ★,2★,High,Medium,1 ★,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1
1,Alan Carvalho,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,Right,"Jan 16, 2015","Dec 31, 2020",€8.5M,€23K,€0,"Dec 31, 2020 On Loan",3 ★,4★,High,Low,2 ★,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2
2,S. Giovinco,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,Right,"Jan 31, 2019",NaN,€9M,€49K,€15.3M,2019 ~ 2022,4 ★,4★,High,Medium,2 ★,73,73+2,73+2,73+2,80+0,79+0,79+0,79+0,80+0,80+0,80+0,80+0,79+1,74+2,74+2,74+2,79+1,59+2,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2
3,J. Evans,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,Right,"Jul 1, 2016",NaN,€275K,€4K,€694K,2016 ~ 2021,2 ★,2★,Medium,Medium,1 ★,7,50+2,50+2,50+2,51+0,51+0,51+0,51+0,51+0,53+2,53+2,53+2,53+2,56+2,56+2,56+2,53+2,56+2,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2
4,Y. Demoncy,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,Right,"Jul 1, 2018",NaN,€725K,€2K,€1.4M,2018 ~ 2021,2 ★,3★,Low,Medium,1 ★,4,56+2,56+2,56+2,59+0,59+0,59+0,59+0,59+0,61+2,61+2,61+2,62+2,63+2,63+2,63+2,62+2,64+2,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,B. Böðvarsson,Iceland,Jagiellonia Białystok,LB,LB,Jagiellonia Białystok 2018 ~ 2021,"6'1""",168lbs,Left,"Feb 8, 2018",NaN,€625K,€3K,€891K,2018 ~ 2021,3 ★,2★,Medium,Medium,1 ★,3,48+2,48+2,48+2,55+0,52+0,52+0,52+0,55+0,54+2,54+2,54+2,58+2,56+2,56+2,56+2,58+2,63+2,60+2,60+2,60+2,63+2,63+2,61+2,61+2,61+2,63+2,16+2
11697,G. Gallon,France,ESTAC Troyes,GK,GK,ESTAC Troyes 2019 ~ 2022,"6'1""",174lbs,Right,"Jul 1, 2019",NaN,€1.6M,€4K,€2.9M,2019 ~ 2022,2 ★,1★,Medium,Medium,1 ★,3,28+2,28+2,28+2,24+0,26+0,26+0,26+0,24+0,24+2,24+2,24+2,25+2,25+2,25+2,25+2,25+2,25+2,26+2,26+2,26+2,25+2,24+2,26+2,26+2,26+2,24+2,69+2
11698,J. Flores,Chile,CD Antofagasta,RM,LM CAM RM,CD Antofagasta 2019 ~ 2024,"5'6""",143lbs,Right,"Jan 23, 2019",NaN,€1.1M,€2K,€1.7M,2019 ~ 2024,3 ★,3★,High,Medium,1 ★,6,62+2,62+2,62+2,67+0,65+0,65+0,65+0,67+0,65+2,65+2,65+2,66+2,58+2,58+2,58+2,66+2,49+2,44+2,44+2,44+2,49+2,45+2,35+2,35+2,35+2,45+2,17+2
11699,Anderson Silva,Brazil,Barnsley,CM,NaN,Barnsley 2010,"6'2""",179lbs,Right,NaN,NaN,€0,€0,€0,2010,3 ★,1★,NaN,NaN,3 ★,3,68+0,68+0,68+0,71+0,68+0,68+0,68+0,71+0,72+0,72+0,72+0,71+0,71+0,71+0,71+0,71+0,66+0,68+0,68+0,68+0,66+0,64+0,60+0,60+0,60+0,64+0,25+0


In [14]:
cateCols = list(df.select_dtypes('object').columns)
cateCols

['name',
 'nationality',
 'club',
 'bp',
 'position',
 'team_&_contract',
 'height',
 'weight',
 'foot',
 'joined',
 'loan_date_end',
 'value',
 'wage',
 'release_clause',
 'contract',
 'w/f',
 'sm',
 'a/w',
 'd/w',
 'ir',
 'hits',
 'ls',
 'st',
 'rs',
 'lw',
 'lf',
 'cf',
 'rf',
 'rw',
 'lam',
 'cam',
 'ram',
 'lm',
 'lcm',
 'cm',
 'rcm',
 'rm',
 'lwb',
 'ldm',
 'cdm',
 'rdm',
 'rwb',
 'lb',
 'lcb',
 'cb',
 'rcb',
 'rb',
 'gk']

In [15]:
#print(df._get_numeric_data().columns)
#print(df.select_dtypes('object').columns)

## Categorical features analysis

In [16]:
# There are some features that seems to be numerical and are show like categorical
# height, weight, joined, loan_date_end, value, wage, release_clause...

In [17]:
possible_nocat_feat = ['height', 'weight', 'joined', 'loan_date_end', 'value', 'wage', 'release_clause', 'w/f', 'sm', 'ir']

In [18]:
# ls	st	rs	lw	lf	cf	rf	rw	lam	cam	ram	lm	lcm	cm	rcm	rm	lwb	ldm	cdm	rdm	rwb	lb	lcb	cb	rcb	rb	gk

In [19]:
df[possible_nocat_feat].head()

,height,weight,joined,loan_date_end,value,wage,release_clause,w/f,sm,ir
0,"5'9""",161lbs,"Jul 1, 2015",NaN,€525K,€4K,€801K,4 ★,2★,1 ★
1,"6'0""",159lbs,"Jan 16, 2015","Dec 31, 2020",€8.5M,€23K,€0,3 ★,4★,2 ★
2,"5'4""",134lbs,"Jan 31, 2019",NaN,€9M,€49K,€15.3M,4 ★,4★,2 ★
3,"5'10""",152lbs,"Jul 1, 2016",NaN,€275K,€4K,€694K,2 ★,2★,1 ★
4,"5'11""",150lbs,"Jul 1, 2018",NaN,€725K,€2K,€1.4M,2 ★,3★,1 ★


### Extract club out of team_&_contrat 

In [20]:
# Drop team_&_contract: After a lot of work team_&_contract is just a duplicated feature

In [21]:
df.drop(columns='team_&_contract',inplace=True)
cateCols.remove('team_&_contract')

In [22]:
# Previous work to drop the columns. There is still a question to be solved

In [23]:
def remove_club(x, club):
    if ( club in x ):
        x = x.replace(club,"").lstrip()
        return x
    else:
        return x

# lambda x, club: x.replace(club,"").lstrip() if (club in x) else x

In [24]:
 # ??? df[['team_&_contract','club']].apply(remove_club) --> pendiente de revisar por Ignacio

### Treat club Nan

In [30]:
df[df['club'].isna()] #==True]

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan_date_end,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova


In [26]:
df['club'] = df['club'].fillna("Available")
# list(map(remove_club, df['team_&_contract'],df['club']))

In [27]:
# df['team_&_contract'] = df['team_&_contract'].apply(lambda x: x.lstrip(df.club)) -> no funciona --> usar map

### Transform Height and Weight

In [28]:
#df.height[0]
#float(df.height[0].rstrip('"').replace('\'','.'))

In [31]:
df.height = df.height.apply(lambda x: float(x.rstrip('"').replace('\'','.')))
df.height

0        5.90
1        6.00
2        5.40
3        5.10
4        5.11
         ... 
11696    6.10
11697    6.10
11698    5.60
11699    6.20
11700    5.70
Name: height, Length: 11701, dtype: float64

In [35]:
#df.weight = df.weight.apply(lambda x: int(x.rstrip('lbs')))
#df.weight

df.weight = list(map(lambda x: int(x.rstrip('lbs')),df.weight))
df.weight

0        161
1        159
2        134
3        152
4        150
        ... 
11696    168
11697    174
11698    143
11699    179
11700    143
Name: weight, Length: 11701, dtype: int64

In [36]:
cateCols.remove('weight')
cateCols.remove('height') # Como se pueden quitar varios valores simultaneamente de una lista?
cateCols

['name',
 'nationality',
 'club',
 'bp',
 'position',
 'foot',
 'joined',
 'loan_date_end',
 'value',
 'wage',
 'release_clause',
 'contract',
 'w/f',
 'sm',
 'a/w',
 'd/w',
 'ir',
 'hits',
 'ls',
 'st',
 'rs',
 'lw',
 'lf',
 'cf',
 'rf',
 'rw',
 'lam',
 'cam',
 'ram',
 'lm',
 'lcm',
 'cm',
 'rcm',
 'rm',
 'lwb',
 'ldm',
 'cdm',
 'rdm',
 'rwb',
 'lb',
 'lcb',
 'cb',
 'rcb',
 'rb',
 'gk']

### Convert joined to date time

In [41]:
df.joined.isna().sum()

41

In [40]:
df[df.joined.isna()]

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan_date_end,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
32,181913,L. Muñoz,25,Mexico,Puebla FC,CB,CB,6.00,168,Right,3,NaN,NaN,€0,€0,€0,2007,151,27,21,53,50,NaN,154,41,NaN,32,35,46,188,65,61,NaN,62,NaN,151,27,NaN,50,53,21,190,69,42.0,39.0,NaN,40,NaN,125,61,64,NaN,68,5,21,35,21,21,1027,285,2 ★,1★,NaN,NaN,1 ★,63,24,39,46,58,55,4,36+0,36+0,36+0,43+0,36+0,36+0,36+0,43+0,44+0,44+0,44+0,43+0,48+0,48+0,48+0,43+0,54+0,52+0,52+0,52+0,54+0,55+0,57+0,57+0,57+0,55+0,21+0,60
44,4233,C. Reyna,33,United States,New York Red Bulls,CM,CM,5.10,157,Right,4,NaN,NaN,€0,€0,€0,2009,271,60,62,66,83,NaN,293,66,NaN,74,74,79,189,65,64,NaN,60,NaN,294,73,NaN,80,72,69,298,69,81.0,70.0,NaN,78,NaN,139,71,68,NaN,71,8,21,74,21,21,1555,420,4 ★,1★,NaN,NaN,2 ★,64,67,74,70,71,74,4,71+0,71+0,71+0,72+0,71+0,71+0,71+0,72+0,74+0,74+0,74+0,72+0,77+0,77+0,77+0,72+0,73+0,75+0,75+0,75+0,73+0,74+0,72+0,72+0,72+0,74+0,24+0,76
470,5589,Figo,35,Portugal,Inter,CF,RW,5.11,165,Right,8,NaN,NaN,€0,€0,€0,2009,293,83,72,54,84,NaN,325,81,NaN,83,76,85,218,73,74,NaN,71,NaN,296,81,NaN,65,71,79,319,73,82.0,82.0,NaN,82,NaN,68,27,41,NaN,72,6,22,76,22,22,1591,430,4 ★,1★,NaN,NaN,5 ★,74,77,82,81,46,70,4,77+0,77+0,77+0,81+0,77+0,77+0,77+0,81+0,79+0,79+0,79+0,81+0,76+0,76+0,76+0,81+0,62+0,70+0,70+0,70+0,62+0,58+0,50+0,50+0,50+0,58+0,24+0,78
664,1615,L. Thuram,35,France,FC Barcelona,CB,CB,6.10,174,Right,4,NaN,NaN,€0,€0,€0,2008,243,67,21,84,71,NaN,239,63,NaN,43,55,78,236,78,73,NaN,85,NaN,232,44,NaN,71,90,27,352,87,90.0,86.0,NaN,89,NaN,167,82,85,NaN,71,8,21,55,21,21,1540,413,3 ★,1★,NaN,NaN,4 ★,75,34,63,72,85,84,6,51+0,51+0,51+0,70+0,51+0,51+0,51+0,70+0,62+0,62+0,62+0,70+0,74+0,74+0,74+0,70+0,78+0,78+0,78+0,78+0,78+0,81+0,83+0,83+0,83+0,81+0,25+0,83
928,11795,A. Stansfield,30,England,Exeter City,ST,ST,5.11,170,Right,6,NaN,NaN,€0,€0,€0,2010,220,50,59,54,57,NaN,206,56,NaN,44,49,57,178,61,61,NaN,56,NaN,221,56,NaN,61,55,49,218,51,53.0,58.0,NaN,56,NaN,84,48,36,NaN,75,6,23,49,23,23,1202,326,3 ★,1★,NaN,NaN,1 ★,61,56,52,56,45,56,4,58+0,58+0,58+0,56+0,58+0,58+0,58+0,56+0,58+0,58+0,58+0,56+0,55+0,55+0,55+0,56+0,52+0,53+0,53+0,53+0,52+0,51+0,50+0,50+0,50+0,51+0,23+0,57
1095,170367,L. Depetris,19,Argentina,Brescia,LW,NaN,5.11,143,Left,2,NaN,NaN,€0,€0,€0,2008,227,69,53,38,67,NaN,258,65,NaN,58,62,73,206,68,72,NaN,66,NaN,223,59,NaN,51,48,65,214,58,39.0,49.0,NaN,68,NaN,86,50,36,NaN,72,6,22,62,22,22,1286,353,3 ★,1★,NaN,NaN,1 ★,70,58,65,68,41,51,3,61+0,61+0,61+0,68+0,61+0,61+0,61+0,68+0,64+0,64+0,64+0,68+0,61+0,61+0,61+0,68+0,58+0,56+0,56+0,56+0,58+0,53+0,47+0,47+0,47+0,53+0,21+0,68
1711,1845,O. Solskjær,34,Norway,Manchester United,ST,ST,5.90,163,Right,8,NaN,NaN,€0,€0,€0,2008,294,60,86,78,70,NaN,264,74,NaN,59,54,77,205,65,64,NaN,76,NaN,254,72,NaN,58,59,65,287,51,71.0,81.0,NaN,84,NaN,55,31,24,NaN,73,10,21,54,21,21,1432,377,4 ★,1★,NaN,NaN,2 ★,64,78,62,75,41,57,7,75+0,75+0,75+0,68+0,75+0,75+0,75+0,68+0,71+0,71+0,71+0,68+0,64+0,64+0,64+0,68+0,52+0,62+0,62+0,62+0,52+0,53+0,49+0,49+0,49+0,53+0,23+0,75
1778,168566,L. Martínez,27,Colombia,Sakaryaspor,GK,GK,6.10,183,Right,5,NaN,NaN,€0,€0,€0,2009,33,9,8,8,8,NaN,143,9,NaN,48,70,16,160,52,56,NaN,52,NaN,150,15,NaN,37,84,14,58,10,NaN,NaN,NaN,48,NaN,17,9,8,NaN,287,69,73,70,69,76,848,411,3 ★,1★,NaN,NaN,2 ★,69,73,70,76,54,69,5,19+0,19+0,19+0,18+0,19+0,19+0,19+0,18+0,24+0,24+0,24+0,18+0,28+0,28+0,28+0,18+0,18+0,21+0,21+0,21+0,

In [ ]:
# pd.to_datetime

In [44]:
df.ova.describe()

count    11701.000000
mean        66.921972
std          6.876081
min         38.000000
25%         62.000000
50%         67.000000
75%         72.000000
max         93.000000
Name: ova, dtype: float64

In [ ]:
# 

In [ ]:
#df.joined = pd.to_datetime(df.joined, erros='coerce')

In [ ]:
#df.joined.isna().sum()

In [56]:
df['joined']=df['joined'].apply(lambda row: pd.to_datetime(row,format='%b %d, %Y'))

In [57]:
df.head()

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan_date_end,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,5.90,161,Right,1,2015-07-01,NaN,525000.0,4000.0,801000.0,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,6.00,159,Right,0,2015-01-16,"Dec 31, 2020",8500000.0,23000.0,0.0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,5.40,134,Right,0,2019-01-31,NaN,9000000.0,49000.0,15300000.0,2019 ~ 2022,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,73,73+2,73+2,73+2,80+0,79+0,79+0,79+0,80+0,80+0,80+0,80+0,79+1,74+2,74+2,74+2,79+1,59+2,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,5.10,152,Right,13,2016-07-01,NaN,275000.0,4000.0,694000.0,2016 ~ 2021,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,7,50+2,50+2,50+2,51+0,51+0,51+0,51+0,51+0,53+2,53+2,53+2,53+2,56+2,56+2,56+2,53+2,56+2,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,5.11,150,Right,8,2018-07-01,NaN,725000.0,2000.0,1400000.0,2018 ~ 2021,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,4,56+2,56+2,56+2,59+0,59+0,59+0,59+0,59+0,61+2,61+2,61+2,62+2,63+2,63+2,63+2,62+2,64+2,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


In [58]:
df[df.joined.isna()]

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan_date_end,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
32,181913,L. Muñoz,25,Mexico,Puebla FC,CB,CB,6.00,168,Right,3,NaT,NaN,0.0,0.0,0.0,2007,151,27,21,53,50,NaN,154,41,NaN,32,35,46,188,65,61,NaN,62,NaN,151,27,NaN,50,53,21,190,69,42.0,39.0,NaN,40,NaN,125,61,64,NaN,68,5,21,35,21,21,1027,285,2 ★,1★,NaN,NaN,1 ★,63,24,39,46,58,55,4,36+0,36+0,36+0,43+0,36+0,36+0,36+0,43+0,44+0,44+0,44+0,43+0,48+0,48+0,48+0,43+0,54+0,52+0,52+0,52+0,54+0,55+0,57+0,57+0,57+0,55+0,21+0,60
44,4233,C. Reyna,33,United States,New York Red Bulls,CM,CM,5.10,157,Right,4,NaT,NaN,0.0,0.0,0.0,2009,271,60,62,66,83,NaN,293,66,NaN,74,74,79,189,65,64,NaN,60,NaN,294,73,NaN,80,72,69,298,69,81.0,70.0,NaN,78,NaN,139,71,68,NaN,71,8,21,74,21,21,1555,420,4 ★,1★,NaN,NaN,2 ★,64,67,74,70,71,74,4,71+0,71+0,71+0,72+0,71+0,71+0,71+0,72+0,74+0,74+0,74+0,72+0,77+0,77+0,77+0,72+0,73+0,75+0,75+0,75+0,73+0,74+0,72+0,72+0,72+0,74+0,24+0,76
470,5589,Figo,35,Portugal,Inter,CF,RW,5.11,165,Right,8,NaT,NaN,0.0,0.0,0.0,2009,293,83,72,54,84,NaN,325,81,NaN,83,76,85,218,73,74,NaN,71,NaN,296,81,NaN,65,71,79,319,73,82.0,82.0,NaN,82,NaN,68,27,41,NaN,72,6,22,76,22,22,1591,430,4 ★,1★,NaN,NaN,5 ★,74,77,82,81,46,70,4,77+0,77+0,77+0,81+0,77+0,77+0,77+0,81+0,79+0,79+0,79+0,81+0,76+0,76+0,76+0,81+0,62+0,70+0,70+0,70+0,62+0,58+0,50+0,50+0,50+0,58+0,24+0,78
664,1615,L. Thuram,35,France,FC Barcelona,CB,CB,6.10,174,Right,4,NaT,NaN,0.0,0.0,0.0,2008,243,67,21,84,71,NaN,239,63,NaN,43,55,78,236,78,73,NaN,85,NaN,232,44,NaN,71,90,27,352,87,90.0,86.0,NaN,89,NaN,167,82,85,NaN,71,8,21,55,21,21,1540,413,3 ★,1★,NaN,NaN,4 ★,75,34,63,72,85,84,6,51+0,51+0,51+0,70+0,51+0,51+0,51+0,70+0,62+0,62+0,62+0,70+0,74+0,74+0,74+0,70+0,78+0,78+0,78+0,78+0,78+0,81+0,83+0,83+0,83+0,81+0,25+0,83
928,11795,A. Stansfield,30,England,Exeter City,ST,ST,5.11,170,Right,6,NaT,NaN,0.0,0.0,0.0,2010,220,50,59,54,57,NaN,206,56,NaN,44,49,57,178,61,61,NaN,56,NaN,221,56,NaN,61,55,49,218,51,53.0,58.0,NaN,56,NaN,84,48,36,NaN,75,6,23,49,23,23,1202,326,3 ★,1★,NaN,NaN,1 ★,61,56,52,56,45,56,4,58+0,58+0,58+0,56+0,58+0,58+0,58+0,56+0,58+0,58+0,58+0,56+0,55+0,55+0,55+0,56+0,52+0,53+0,53+0,53+0,52+0,51+0,50+0,50+0,50+0,51+0,23+0,57
1095,170367,L. Depetris,19,Argentina,Brescia,LW,NaN,5.11,143,Left,2,NaT,NaN,0.0,0.0,0.0,2008,227,69,53,38,67,NaN,258,65,NaN,58,62,73,206,68,72,NaN,66,NaN,223,59,NaN,51,48,65,214,58,39.0,49.0,NaN,68,NaN,86,50,36,NaN,72,6,22,62,22,22,1286,353,3 ★,1★,NaN,NaN,1 ★,70,58,65,68,41,51,3,61+0,61+0,61+0,68+0,61+0,61+0,61+0,68+0,64+0,64+0,64+0,68+0,61+0,61+0,61+0,68+0,58+0,56+0,56+0,56+0,58+0,53+0,47+0,47+0,47+0,53+0,21+0,68
1711,1845,O. Solskjær,34,Norway,Manchester United,ST,ST,5.90,163,Right,8,NaT,NaN,0.0,0.0,0.0,2008,294,60,86,78,70,NaN,264,74,NaN,59,54,77,205,65,64,NaN,76,NaN,254,72,NaN,58,59,65,287,51,71.0,81.0,NaN,84,NaN,55,31,24,NaN,73,10,21,54,21,21,1432,377,4 ★,1★,NaN,NaN,2 ★,64,78,62,75,41,57,7,75+0,75+0,75+0,68+0,75+0,75+0,75+0,68+0,71+0,71+0,71+0,68+0,64+0,64+0,64+0,68+0,52+0,62+0,62+0,62+0,52+0,53+0,49+0,49+0,49+0,53+0,23+0,75
1778,168566,L. Martínez,27,Colombia,Sakaryaspor,GK,GK,6.10,183,Right,5,NaT,NaN,0.0,0.0,0.0,2009,33,9,8,8,8,NaN,143,9,NaN,48,70,16,160,52,56,NaN,52,NaN,150,15,NaN,37,84,14,58,10,NaN,NaN,NaN,48,NaN,17,9,8,NaN,287,69,73,70,69,76,848,411,3 ★,1★,NaN,NaN,2 ★,69,73,70,76,54,69,5,19+0,19+0,19+0,18+0,19+0,19+0,19+0,18+0,24+0,24+0,24+0,18+0,28+0,28+0,28+0,1

### Remove stars

In [74]:
def starout(x):
    if ('★'in x):
        x = x.replace('★',"")
        x = int(x)
        return x


In [165]:
#map(starout,w/f)


NameError: name 'w' is not defined

In [78]:
#df.sm = df.sm.apply(starout)
df.sm.unique()

array([2, 4, 3, 1, 5])

In [79]:
df.ir = df.ir.apply(starout)
df.ir.unique()

array([1, 2, 3, 5, 4])

In [80]:
df['w/f'] = df['w/f'].apply(starout)
df['w/f'].unique()

array([4, 3, 2, 5, 1])

In [ ]:
#map(starout,sm)
#map((starout,ir)

### Convert value, wage and release_clause to numerical

In [46]:
# value	wage	release_clause

# define the function to apply to the columns

def standard_amount(x):
    if x[-1] == 'K':
        x = float(x.lstrip('€').rstrip('K'))*1000
        return x
    elif x[-1] == 'M':
        x = float(x.lstrip('€').rstrip('M'))*1000000
        return x
    else:
        x = float(x.lstrip('€'))
        return x


In [47]:
# isinstance(x,str) --> type(x) == str

moneyCols = []
for c in df.columns:
    if isinstance(df[c][0],str) and df[c][0][0] == '€':
        moneyCols.append(c)

moneyCols    

['value', 'wage', 'release_clause']

In [48]:
df[moneyCols]

,value,wage,release_clause
0,€525K,€4K,€801K
1,€8.5M,€23K,€0
2,€9M,€49K,€15.3M
3,€275K,€4K,€694K
4,€725K,€2K,€1.4M
...,...,...,...
11696,€625K,€3K,€891K
11697,€1.6M,€4K,€2.9M
11698,€1.1M,€2K,€1.7M
11699,€0,€0,€0


In [49]:
for f in moneyCols:
    df[f] = df[f].apply(standard_amount)

df[moneyCols]

,value,wage,release_clause
0,525000.0,4000.0,801000.0
1,8500000.0,23000.0,0.0
2,9000000.0,49000.0,15300000.0
3,275000.0,4000.0,694000.0
4,725000.0,2000.0,1400000.0
...,...,...,...
11696,625000.0,3000.0,891000.0
11697,1600000.0,4000.0,2900000.0
11698,1100000.0,2000.0,1700000.0
11699,0.0,0.0,0.0


### Converting player's positions' features

In [ ]:
#for col in ['ls','st','rs','lw',lf	cf	rf	rw	lam	cam	ram	lm	lcm	cm	rcm	rm	lwb	ldm	cdm	rdm	rwb	lb	lcb	cb	rcb	rb	gk]
#    df[col] = list(map(lambda x: int(x.split("+")[0]) + int(x.split("+")[1], df[col]))
                   

In [63]:
df.head(1)

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan_date_end,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,5.9,161,Right,1,2015-07-01,NaN,525000.0,4000.0,801000.0,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64


In [55]:
#i0 = list(df.columns).index('ls')
#positionsCols = index

72

In [64]:
df[cateCols].head(1)

,name,nationality,club,bp,position,foot,joined,loan_date_end,value,wage,release_clause,contract,w/f,sm,a/w,d/w,ir,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,A. Pasche,Switzerland,FC Lausanne-Sport,CM,CM CDM,Right,2015-07-01,NaN,525000.0,4000.0,801000.0,2015 ~ 2020,4 ★,2★,High,Medium,1 ★,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1


In [67]:
# Get the names of the columns of the positions:
positionCols = []
for c in df.columns:
    if isinstance(df[c][0],str) and '+' in df[c][0]:
        positionCols.append(c)
positionCols

['ls',
 'st',
 'rs',
 'lw',
 'lf',
 'cf',
 'rf',
 'rw',
 'lam',
 'cam',
 'ram',
 'lm',
 'lcm',
 'cm',
 'rcm',
 'rm',
 'lwb',
 'ldm',
 'cdm',
 'rdm',
 'rwb',
 'lb',
 'lcb',
 'cb',
 'rcb',
 'rb',
 'gk']

In [70]:
for col in positionCols:
    df[col] = list(map(lambda x: int(x.split("+")[0]) + int(x.split("+")[1]), df[col]))
df.head()

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan_date_end,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,5.90,161,Right,1,2015-07-01,NaN,525000.0,4000.0,801000.0,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,59,59,59,61,62,62,62,61,64,64,64,64,64,64,64,64,60,60,60,60,60,59,55,55,55,59,16,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,6.00,159,Right,0,2015-01-16,"Dec 31, 2020",8500000.0,23000.0,0.0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77,77,77,77,77,77,77,77,77,77,77,77,70,70,70,77,59,55,55,55,59,55,50,50,50,55,20,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,5.40,134,Right,0,2019-01-31,NaN,9000000.0,49000.0,15300000.0,2019 ~ 2022,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,73,75,75,75,80,79,79,79,80,80,80,80,80,76,76,76,80,61,58,58,58,61,55,43,43,43,55,14,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,5.10,152,Right,13,2016-07-01,NaN,275000.0,4000.0,694000.0,2016 ~ 2021,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,7,52,52,52,51,51,51,51,51,55,55,55,55,58,58,58,55,58,60,60,60,58,59,60,60,60,59,16,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,5.11,150,Right,8,2018-07-01,NaN,725000.0,2000.0,1400000.0,2018 ~ 2021,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,4,58,58,58,59,59,59,59,59,63,63,63,64,65,65,65,64,66,66,66,66,66,65,63,63,63,65,17,65


In [72]:
df._get_numeric_data()

,id,age,height,weight,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,pac,sho,pas,dri,def,phy,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,26,5.90,161,1,525000.0,4000.0,801000.0,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,69,51,63,63,51,60,59,59,59,61,62,62,62,61,64,64,64,64,64,64,64,64,60,60,60,60,60,59,55,55,55,59,16,64
1,188044,30,6.00,159,0,8500000.0,23000.0,0.0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,83,75,68,82,33,71,77,77,77,77,77,77,77,77,77,77,77,77,70,70,70,77,59,55,55,55,59,55,50,50,50,55,20,77
2,184431,33,5.40,134,0,9000000.0,49000.0,15300000.0,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,80,77,78,86,27,56,75,75,75,80,79,79,79,80,80,80,80,80,76,76,76,80,61,58,58,58,61,55,43,43,43,55,14,80
3,233796,22,5.10,152,13,275000.0,4000.0,694000.0,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,57,44,54,57,57,60,52,52,52,51,51,51,51,51,55,55,55,55,58,58,58,55,58,60,60,60,58,59,60,60,60,59,16,59
4,234799,23,5.11,150,8,725000.0,2000.0,1400000.0,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,66,44,60,64,60,66,58,58,58,59,59,59,59,59,63,63,63,64,65,65,65,64,66,66,66,66,66,65,63,63,63,65,17,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,25,6.10,168,5,625000.0,3000.0,891000.0,224,62,24,51,59,28.0,248,57,49.0,29,54,59,333,78,73,68.0,53,61.0,273,31,72.0,77,68,25,266,63,59.0,51.0,54.0,39,46.0,184,59,64,61.0,61,13,14,14,6,14,1589,347,75,28,56,59,60,69,50,50,50,55,52,52,52,55,56,56,56,60,58,58,58,60,65,62,62,62,65,65,63,63,63,65,18,65
11697,214680,27,6.10,174,4,1600000.0,4000.0,2900000.0,84,12,14,13,29,16.0,96,17,15.0,19,27,18,237,40,40,38.0,65,54.0,217,50,59.0,29,64,15,101,30,22.0,15.0,15.0,19,56.0,41,12,16,13.0,347,72,67,66,70,72,1123,387,72,67,66,72,40,70,30,30,30,24,26,26,26,24,26,26,26,27,27,27,27,27,27,28,28,28,27,26,28,28,28,26,71,70
11698,221489,22,5.60,143,8,1100000.0,2000.0,1700000.0,286,64,66,51,64,41.0,291,71,57.0,38,58,67,388,77,76,86.0,62,87.0,278,61,71.0,64,37,45,241,27,25.0,66.0,58.0,65,58.0,62,37,12,13.0,57,13,10,11,7,16,1603,337,76,60,60,72,26,43,64,64,64,67,65,65,65,67,67,67,67,68,60,60,60,68,51,46,46,46,51,47,37,37,37,47,19,67
11699,146717,26,6.20,179,7,0.0,0.0,0.0,254,64,66,51,73,NaN,284,74,NaN,63,72,75,198,71,72,NaN,55,NaN,258,61,NaN,71,64,62,241,64,61.0,63.0,NaN,53,NaN,113,55,58,NaN,81,9,24,72,24,24,1429,400,72,63,70,72,57,66,68,68,68,71,68,68,68,71,72,72,72,71,71,71,71,71,66,68,68,68,66,64,60,60,60,64,25,68


### Check contract and moneyCols == 0

In [81]:
# Contract has 362 unique values, because of lack of time we are not treating that column and droping it

df = df.drop(columns=['contract'])
df.head()

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,joined,loan_date_end,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,5.90,161,Right,1,2015-07-01,NaN,525000.0,4000.0,801000.0,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4,2,High,Medium,1,69,51,63,63,51,60,3,59,59,59,61,62,62,62,61,64,64,64,64,64,64,64,64,60,60,60,60,60,59,55,55,55,59,16,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,6.00,159,Right,0,2015-01-16,"Dec 31, 2020",8500000.0,23000.0,0.0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3,4,High,Low,2,83,75,68,82,33,71,44,77,77,77,77,77,77,77,77,77,77,77,77,70,70,70,77,59,55,55,55,59,55,50,50,50,55,20,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,5.40,134,Right,0,2019-01-31,NaN,9000000.0,49000.0,15300000.0,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4,4,High,Medium,2,80,77,78,86,27,56,73,75,75,75,80,79,79,79,80,80,80,80,80,76,76,76,80,61,58,58,58,61,55,43,43,43,55,14,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,5.10,152,Right,13,2016-07-01,NaN,275000.0,4000.0,694000.0,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2,2,Medium,Medium,1,57,44,54,57,57,60,7,52,52,52,51,51,51,51,51,55,55,55,55,58,58,58,55,58,60,60,60,58,59,60,60,60,59,16,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,5.11,150,Right,8,2018-07-01,NaN,725000.0,2000.0,1400000.0,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2,3,Low,Medium,1,66,44,60,64,60,66,4,58,58,58,59,59,59,59,59,63,63,63,64,65,65,65,64,66,66,66,66,66,65,63,63,63,65,17,65


In [82]:
# df[moneyCols][df.wage == 0]

,value,wage,release_clause
32,0.0,0.0,0.0
44,0.0,0.0,0.0
102,0.0,0.0,0.0
105,0.0,0.0,0.0
137,0.0,0.0,0.0
...,...,...,...
11648,0.0,0.0,0.0
11650,0.0,0.0,0.0
11679,0.0,0.0,0.0
11680,0.0,0.0,0.0


In [83]:
len(df[moneyCols][df.wage == 0])

300

## Deal with NaN

In [105]:
#df.isna().sum()[df.isna().sum() != 0]
#nan_df = pd.Dataframe(df.isna().sum()[df.isna().sum() != 0]) #, c

In [45]:
nan_df = pd.DataFrame(df.isna().sum(), columns=['NaN'])

nanCols = list(nan_df[nan_df.NaN != 0].index)

#nanCols
nan_df.T[nanCols]

,position,joined,loan_date_end,volleys,curve,agility,balance,jumping,interceptions,positioning,vision,composure,sliding_tackle,a/w,d/w
NaN,288,41,11072,41,41,41,41,41,7,7,41,279,41,59,59


In [107]:
#nanCols = list(df.isna().sum()[df.isna().sum() != 0].index)
#nanCols

In [108]:
# Percentage of NaN

In [109]:
nan_df = pd.DataFrame(round(df[nanCols].isna().sum()/len(df),4)*100, columns=['percent_nan'])
nan_df.T

,club,position,joined,loan_date_end,volleys,curve,agility,balance,jumping,interceptions,positioning,vision,composure,sliding_tackle,a/w,d/w
percent_nan,0.15,2.46,0.35,94.62,0.35,0.35,0.35,0.35,0.35,0.06,0.06,0.35,2.38,0.35,0.5,0.5


In [110]:
# Drop loan_date_end variable
df.drop(columns='loan_date_end', inplace=True)
nanCols.remove('loan_date_end')

In [111]:
# Personal question: check if the 41 NaN values coincide in the same rows 
df.volleys[df.joined.isna()].value_counts()
#==df.volleys.isna().iloc[0]
# df.joined.isna()==df.volleys.isna()[df.joined.isna()==df.volleys.isna() == True]

Series([], Name: volleys, dtype: int64)

In [112]:
nan41cols = df.isna().sum()[df.isna().sum() == 41]
nan41cols

joined            41
volleys           41
curve             41
agility           41
balance           41
jumping           41
vision            41
sliding_tackle    41
dtype: int64

In [113]:
# Drop all rows with NaN values
dfc = df.dropna(axis=0)
dfc.shape

(11211, 100)

In [116]:
len(df)

11701

In [115]:
dfc.isna().sum()[dfc.isna().sum() != 0]

Series([], dtype: int64)

In [117]:
# We have dropped a 4.18% of rows